In [1]:
import os
import pandas as pd
import numpy as np
import re
import unicodedata
import sys

In [2]:
def validname(value):
    """
    Normalizes string, converts to lowercase, removes non-alpha characters,
    and converts spaces to hyphens.
    """
    value = re.sub('[^\w\s-]', '', value).strip().lower()
    value = re.sub('[-\s]+', '-', value)
    return value

In [3]:
art = pd.read_csv('../data/ALL_ARTWORKS.csv',low_memory=False)

In [4]:
# Transform bad collections
art.loc[art['collection'] == "Musée national d'art moderne / Centre de cr��ation industrielle", 'collection'] = "Musée national d'art moderne / Centre de création industrielle"
art.loc[art['collection'] == "Mus��e d'Art moderne et contemporain de la Ville de Strasbourg", 'collection'] = "Musée d'Art moderne et contemporain de la Ville de Strasbourg"
art.loc[art['collection'] == "Mus��e national d'art moderne / Centre de création industrielle", 'collection'] = "Musée national d'art moderne / Centre de création industrielle"
art.loc[art['collection'] == "La Piscine, Mus��e d'art et d'industrie André Diligent, Roubaix", 'collection'] = "La Piscine, Musée d'art et d'industrie André Diligent, Roubaix"
art.loc[art['collection'] == "Mus��e d'art contemporain de Lyon", 'collection'] = "Musée d'art contemporain de Lyon"

In [5]:
collections = sorted(art['collection'].dropna().unique())
fracs = []
order = []
for museum in collections:
    if "musée national d'art moderne" in museum.lower():
        order.append(0)
    elif "centre national des arts plastiques" in museum.lower():
        order.append(1)
    elif " national" in museum.lower():
        order.append(2)
    elif "frac" in museum.lower():
        fracs.append(museum)
        order.append(3)
    else:
        order.append(4)
ordercol = pd.DataFrame({'coll': collections, 'ord': order})
ordercol['lower'] = ordercol['coll'].str.lower()
ordercol = ordercol.sort_values(by=["ord", "lower"])
collections = ordercol["coll"].tolist()

In [6]:
stylefile = open("style.css", "r")
style = stylefile.read()

In [7]:
begin = """
<!DOCTYPE html>
<html>
<head>
  <title>Rapport Art & Genre</title>
  <link href="https://fonts.googleapis.com/css?family=Roboto:400,700" rel="stylesheet">
  <meta charset="utf-8"/>
  <link rel="stylesheet" type="text/css" href="style.css">
</head>
"""

In [8]:
main = ""

In [9]:
pagestring = """
    <!-- begin page -->
    <div class="page">
    <div class="column">
      <!-- top left -->
      <div class="cell cartel">
        <div class="cartel-content">
          <h1> %s </h1>
          <ul>
              <li>
                Période étudiée : 1945 – 2017
              </li>
              <li>
                <span class="legend-item women">
                  <div>
                      femmes
                  </div>
                </span>
                <span class="legend-item men">
                  <div>
                      hommes
                  </div>
                </span>
              </li>
          </ul>
        </div>
      </div>
      <!-- middle left -->
      <div class="cell">
        <div class="figure-container">
          <h3 class="figure-title">Âge de l'artiste à l'acquisition</h3>
          <img src="%s"></img>
        </div>
      </div>
      <!-- bottom left -->
      <div class="cell">
        <div class="figure-container">
          <h3 class="figure-title">Modalités d'acquisition</h3>
          <img src="%s"></img>
        </div>
      </div>
    </div>
    <div class="column">
      <!-- top right -->
      <div class="cell">
        <div class="figure-container">
          <h3 class="figure-title">Acquisitions totales</h3>
          <img src="%s"></img>
        </div>
      </div>
      <!-- middle right -->
      <div class="cell">
        <div class="figure-container">
          <h3 class="figure-title">Acquisition d'artistes français.e.s</h3>
          <img src="%s"></img>
        </div>
      </div>
      <!-- bottom right -->
      <div class="cell">
        <div class="figure-container">
          <h3 class="figure-title">Acquisitions par domaine</h3>
          <img src="%s"></img>
        </div>
      </div>
    </div>
    </div>
    <!-- end page -->
    """

In [10]:
# Global
folder = "_global"
fig_age = "figures/" + folder + "/age_acquisition.png"
fid_mod = "figures/" + folder + "/ratio_acquisitions.png"
fig_ove = "figures/" + folder + "/ratio_overall.png"
fig_nat = "figures/" + folder + "/ratio_nationality.png"
fig_dom = "figures/" + folder + "/ratio_domains.png"
page = pagestring % ("Ensemble des collections", fig_age, fid_mod, fig_ove, fig_nat, fig_dom)
main = main + page

In [11]:
# FRACs
folder = "_fracs"
fig_age = "figures/" + folder + "/age_acquisition.png"
fid_mod = "figures/" + folder + "/ratio_acquisitions.png"
fig_ove = "figures/" + folder + "/ratio_overall.png"
fig_nat = "figures/" + folder + "/ratio_nationality.png"
fig_dom = "figures/" + folder + "/ratio_domains.png"
page = pagestring % ("Ensemble des FRAC", fig_age, fid_mod, fig_ove, fig_nat, fig_dom)
main = main + page

In [12]:
# Collections nationales
# folder = "_nationales"
# fig_age = "figures/" + folder + "/age_acquisition.png"
# fid_mod = "figures/" + folder + "/ratio_acquisitions.png"
# fig_ove = "figures/" + folder + "/ratio_overall.png"
# fig_nat = "figures/" + folder + "/ratio_nationality.png"
# fig_dom = "figures/" + folder + "/ratio_domains.png"
# page = pagestring % ("Ensemble des musées nationaux", fig_age, fid_mod, fig_ove, fig_nat, fig_dom)
# main = main + page

In [13]:
#Autres collections
for col in collections:
    folder = validname(col)
    fig_age = "figures/" + folder + "/age_acquisition.png"
    fid_mod = "figures/" + folder + "/ratio_acquisitions.png"
    fig_ove = "figures/" + folder + "/ratio_overall.png"
    fig_nat = "figures/" + folder + "/ratio_nationality.png"
    fig_dom = "figures/" + folder + "/ratio_domains.png"
    
    page = pagestring % (col, fig_age, fid_mod, fig_ove, fig_nat, fig_dom)
    main = main + page

In [14]:
body = """
<body>
%s
</body>
""" % main

In [15]:
content = begin + body

In [16]:
html_file = open("presentation.html", "w", encoding="utf-8")
html_file.write(content)
html_file.close()